<a href="https://colab.research.google.com/github/2303A51553/Natural-language-process/blob/main/BErt2bert.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [1]:
!pip install tensorflow nltk numpy pandas sacrebleu


     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 51.8/51.8 kB 2.7 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 104.1/104.1 kB 4.6 MB/s eta 0:00:00


In [5]:
import re
import string
import numpy as np
import pandas as pd
import tensorflow as tf
from tensorflow import keras
from tensorflow.keras import layers
import nltk
from nltk.tokenize import word_tokenize
from nltk.corpus import stopwords

nltk.download('punkt')
nltk.download('stopwords')
nltk.download('punkt_tab') # Added to resolve the LookupError
df = pd.read_csv("/content/Legal_Summarisation_100_Final (1).csv")

stop_words = set(stopwords.words('english'))

def preprocess_text(text):
    text = str(text).lower()
    text = text.translate(str.maketrans('', '', string.punctuation))
    text = re.sub(r'[^a-zA-Z0-9\s]', '', text)
    tokens = word_tokenize(text)
    tokens = [w for w in tokens if w not in stop_words]
    return " ".join(tokens)


df["clean_doc"] = df["document"].apply(preprocess_text)
df["clean_sum"] = df["summary"].apply(preprocess_text)


[nltk_data] Downloading package punkt to /root/nltk_data...
[nltk_data]   Package punkt is already up-to-date!
[nltk_data] Downloading package stopwords to /root/nltk_data...
[nltk_data]   Package stopwords is already up-to-date!
[nltk_data] Downloading package punkt_tab to /root/nltk_data...
[nltk_data]   Unzipping tokenizers/punkt_tab.zip.


In [6]:
max_vocab = 30000
max_enc_len = 256
max_dec_len = 64

vectorizer = layers.TextVectorization(
    max_tokens=max_vocab,
    output_mode="int",
    output_sequence_length=max_enc_len,
    standardize=None
)

vectorizer.adapt(df["clean_doc"].tolist() + df["clean_sum"].tolist())

vocab = vectorizer.get_vocabulary()
vocab_size = len(vocab)


In [7]:
class PositionalEmbedding(layers.Layer):
    def __init__(self, vocab_size, embed_dim, max_len):
        super().__init__()
        self.token_emb = layers.Embedding(vocab_size, embed_dim, mask_zero=True)
        self.pos_emb   = layers.Embedding(max_len, embed_dim)

    def call(self, x):
        max_len = tf.shape(x)[1]
        positions = tf.range(start=0, limit=max_len)
        pos_embeddings = self.pos_emb(positions)
        tok_embeddings = self.token_emb(x)
        return tok_embeddings + pos_embeddings


In [8]:
def bert_encoder_layer(embed_dim, num_heads, ff_dim):
    inputs = layers.Input(shape=(None, embed_dim))
    attn = layers.MultiHeadAttention(num_heads=num_heads, key_dim=embed_dim)(inputs, inputs)
    out1 = layers.LayerNormalization()(inputs + attn)

    ffn = layers.Dense(ff_dim, activation="relu")(out1)
    ffn = layers.Dense(embed_dim)(ffn)
    out2 = layers.LayerNormalization()(out1 + ffn)

    return keras.Model(inputs, out2)


In [9]:
def bert_decoder_layer(embed_dim, num_heads, ff_dim):
    dec_in = layers.Input(shape=(None, embed_dim))
    enc_out = layers.Input(shape=(None, embed_dim))

    attn1 = layers.MultiHeadAttention(num_heads=num_heads, key_dim=embed_dim)(dec_in, dec_in, use_causal_mask=True)
    out1 = layers.LayerNormalization()(dec_in + attn1)

    attn2 = layers.MultiHeadAttention(num_heads=num_heads, key_dim=embed_dim)(out1, enc_out)
    out2 = layers.LayerNormalization()(out1 + attn2)

    ffn = layers.Dense(ff_dim, activation="relu")(out2)
    ffn = layers.Dense(embed_dim)(ffn)
    out3 = layers.LayerNormalization()(out2 + ffn)

    return keras.Model([dec_in, enc_out], out3)


In [10]:
embed_dim=256
ff_dim=512
num_heads=8
num_layers=4

# Encoder
enc_input=keras.Input(shape=(None,), dtype="int32")
enc_emb=PositionalEmbedding(vocab_size, embed_dim, max_enc_len)(enc_input)
encoder=enc_emb
for _ in range(num_layers):
    encoder=bert_encoder_layer(embed_dim, num_heads, ff_dim)(encoder)

# Decoder
dec_input = keras.Input(shape=(None,), dtype="int32")
dec_emb = PositionalEmbedding(vocab_size, embed_dim, max_dec_len)(dec_input)
decoder = dec_emb
for _ in range(num_layers):
    decoder = bert_decoder_layer(embed_dim, num_heads, ff_dim)([decoder, encoder])

# Output to vocab
final_output = layers.Dense(vocab_size)(decoder)

model = keras.Model([enc_input, dec_input], final_output)
model.summary()


Model: "functional_8"

┏━━━━━━━━━━━━━━━━━━━━━┳━━━━━━━━━━━━━━━━━━━┳━━━━━━━━━━━━┳━━━━━━━━━━━━━━━━━━━┓
┃ Layer (type)        ┃ Output Shape      ┃    Param # ┃ Connected to      ┃
┡━━━━━━━━━━━━━━━━━━━━━╇━━━━━━━━━━━━━━━━━━━╇━━━━━━━━━━━━╇━━━━━━━━━━━━━━━━━━━┩
│ input_layer         │ (None, None)      │          0 │ -                 │
│ (InputLayer)        │                   │            │                   │
├─────────────────────┼───────────────────┼────────────┼───────────────────┤
│ positional_embeddi… │ (None, None, 256) │     88,576 │ input_layer[0][0] │
│ (PositionalEmbeddi… │                   │            │                   │
├─────────────────────┼───────────────────┼────────────┼───────────────────┤
│ functional          │ (None, None, 256) │  2,367,488 │ positional_embed… │
│ (Functional)        │                   │            │                   │
├─────────────────────┼───────────────────┼────────────┼───────────────────┤
│ functional_1        │ (None, None, 256) │  2,367,488 │ functional[0][0]  │
│ (Functional)        │                   │            │                   │
├─────────────────────┼───────────────────┼────────────┼───────────────────┤
│ input_layer_5       │ (None, None)      │          0 │ -                 │
│ (InputLayer)        │                   │            │                   │
├─────────────────────┼───────────────────┼────────────┼───────────────────┤
│ functional_2        │ (None, None, 256) │  2,367,488 │ functional_1[0][… │
│ (Functional)        │                   │            │                   │
├─────────────────────┼───────────────────┼────────────┼───────────────────┤
│ positional_embeddi… │ (None, None, 256) │     39,424 │ input_layer_5[0]… │
│ (PositionalEmbeddi… │                   │            │                   │
├─────────────────────┼───────────────────┼────────────┼───────────────────┤
│ functional_3        │ (None, None, 256) │  2,367,488 │ functional_2[0][… │
│ (Functional)        │                   │            │                   │
├─────────────────────┼───────────────────┼────────────┼───────────────────┤
│ functional_4        │ (None, None, 256) │  4,471,552 │ positional_embed… │
│ (Functional)        │                   │            │ functional_3[0][… │
├─────────────────────┼───────────────────┼────────────┼───────────────────┤
│ functional_5        │ (None, None, 256) │  4,471,552 │ functional_4[0][… │
│ (Functional)        │                   │            │ functional_3[0][… │
├─────────────────────┼───────────────────┼────────────┼───────────────────┤
│ functional_6        │ (None, None, 256) │  4,471,552 │ functional_5[0][… │
│ (Functional)        │                   │            │ functional_3[0][… │
├─────────────────────┼───────────────────┼────────────┼───────────────────┤
│ functional_7        │ (None, None, 256) │  4,471,552 │ functional_6[0][… │
│ (Functional)        │                   │            │ functional_3[0][… │
├─────────────────────┼───────────────────┼────────────┼───────────────────┤
│ dense_16 (Dense)    │ (None, None, 90)  │     23,130 │ functional_7[0][… │
└─────────────────────┴───────────────────┴────────────┴───────────────────┘

 Total params: 27,507,290 (104.93 MB)

 Trainable params: 27,507,290 (104.93 MB)

 Non-trainable params: 0 (0.00 B)

In [13]:
enc = vectorizer(df["clean_doc"].tolist()).numpy()

# Tokenize summaries and then truncate to max_dec_len
dec_raw=vectorizer(df["clean_sum"].tolist()).numpy()
dec=dec_raw[:, :max_dec_len] # Ensure decoder sequences are max_dec_len long

dec_in=np.concatenate([np.zeros((dec.shape[0],1)), dec[:,:-1]], axis=1)
dec_out=dec


In [14]:
model.compile(
    optimizer="adam",
    loss=keras.losses.SparseCategoricalCrossentropy(from_logits=True)
)

model.fit([enc, dec_in], dec_out, epochs=5, batch_size=4)


Epoch 1/5
25/25 ━━━━━━━━━━━━━━━━━━━━ 154s 5s/step - loss: 2.4791
Epoch 2/5
25/25 ━━━━━━━━━━━━━━━━━━━━ 118s 4s/step - loss: 0.6604
Epoch 3/5
25/25 ━━━━━━━━━━━━━━━━━━━━ 93s 4s/step - loss: 0.6465
Epoch 4/5
25/25 ━━━━━━━━━━━━━━━━━━━━ 142s 4s/step - loss: 0.6547
Epoch 5/5
25/25 ━━━━━━━━━━━━━━━━━━━━ 140s 4s/step - loss: 0.6464


In [17]:
id_to_word={i:w for i,w in enumerate(vocab)}

def generate(text):
    e=vectorizer([text]).numpy()
    d=np.zeros((1, max_dec_len), dtype=int)

    for i in range(1, max_dec_len):
        preds=model.predict([e, d[:,:i]], verbose=0)
        next_id=np.argmax(preds[0, i-1])
        d[0, i]=next_id
        if next_id==0:
            break

    words=[id_to_word.get(i,"") for i in d[0] if i!=0]
    return " ".join(words)


In [16]:
import sacrebleu

preds=[]
refs =[]

for i in range(30):
    p=generate(df["clean_doc"].iloc[i])
    preds.append(p)
    refs.append(df["clean_sum"].iloc[i])

bleu = sacrebleu.corpus_bleu(preds, [refs])
print("BLEU:", bleu.score)


BLEU: 0.0
